### Applying Deep CNN on AffectNet Dataset [http://mohammadmahoor.com/affectnet/] 
### Trained on 6 different emotions

In [ ]:
# ###  If the problem persists uncomment the fllowing lines and then run...

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

# 
# !nvidia-smi -h

# !nvidia-smi -q -d UTILIZATION

import os

# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"
# # print(CUDA_VISIBLE_DEVICES)
# import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# for physical_device in physical_devices:
#     tf.config.experimental.set_memory_growth(physical_device, True)


In [ ]:
# !pip install keras
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# from mlxtend.plotting import plot_confusion_matrix
# from keras import models
from tensorflow.keras.utils import to_categorical
# from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, LSTM, merge
from tensorflow.keras.models import Model
# import keras
import csv
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Activation,BatchNormalization, Flatten, Conv2D, AveragePooling2D
from tqdm import tqdm
import numpy as np # linear algebra
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
# from imblearn.over_sampling import RandomOverSampler
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
# from tensorflow.keras.layers import Dropout
# import tensorflow
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.layers import Dropout, MaxPooling2D

In [ ]:
TrainDataGen = ImageDataGenerator(
                rotation_range = 5,          ## rotated image training
                width_shift_range = 0.1,
                height_shift_range = 0.1,   ## shifted image training
                rescale = 1.0/255,
                shear_range = 0.2,
                #zoom_range = 0.2,		   ## zoomed image training
                #horizontal_flip = False,
                fill_mode = 'nearest')

In [ ]:
trainGenerator = TrainDataGen.flow_from_directory(
    
                '/home/ug2019/eee/19085089/EmojiGeneration/Affectnet_Dataset/Train_set',
#                 '/content/drive/MyDrive/emoji/train_set',##ENTER THE PATH BOTH THE FOLDER OF TRAIN IMAGE .
                target_size = (224, 224),    # enter the correct size of image
                batch_size = 32,  # will change it
                # color_mode = "grayscale",
    
			          class_mode = "categorical")




Found 255217 images belonging to 6 classes.


In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = valid_datagen.flow_from_directory(
                '/home/ug2019/eee/19085089/EmojiGeneration/Affectnet_Dataset/val_set',   ## directory of folder having validation set,
                target_size=(224,224),
                batch_size=32,
                # color_mode = "grayscale",
                class_mode= 'categorical')


Found 2500 images belonging to 6 classes.


In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# tf.config.list_physical_devices(device_type='GPU')
from tensorflow.keras import optimizers


In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy(    
#     devices = [
#         "/gpu:0",
#         "/gpu:1",
#         "/gpu:2",
#         "/gpu:3",
#         "/gpu:4"
#               ]
)

# mirrored_strategy = tf.distribute.MirroredStrategy(    
#     devices = [
#         "/gpu:0",
#         "/gpu:1",
#         "/gpu:2",
# #         "/gpu:3",
# #         "/gpu:4"
#               ]

# )

'''
Avalilable regulizers : - l1, l2, l1_l2
'''



with mirrored_strategy.scope():
    model = tf.keras.Sequential(

        [

            layers.Conv2D(64, (3,3), input_shape = (224,224,3), strides = 1, padding = "same"),
            layers.BatchNormalization(),
            layers.Activation(activation='relu'),
            layers.Dropout(0.5),
            layers.Conv2D(128, (3,3), strides = 1, padding = "same", kernel_regularizer='l1_l2', bias_regularizer='l1_l2'),
            layers.BatchNormalization(),
            layers.Activation(activation='relu'),
            layers.MaxPooling2D(pool_size = (2,2)),
            layers.Dropout(0.5),
            layers.Conv2D(512, (3,3), strides = 1,padding = "same", kernel_regularizer='l1_l2', bias_regularizer='l1_l2'),
            layers.BatchNormalization(),
            layers.Activation(activation='relu'),
            layers.MaxPooling2D(pool_size = (2,2)),
            layers.Dropout(0.5),
            layers.Conv2D(512, (3,3), strides = 1,padding = "same", kernel_regularizer='l1_l2', bias_regularizer='l1_l2'),
            layers.BatchNormalization(),
            layers.Activation(activation='relu'),
            layers.MaxPooling2D(pool_size = (2,2)),
            layers.Dropout(0.5),
            layers.Flatten(),
            layers.Dense(256, activation = "relu", kernel_initializer = "uniform"),  #c check for kernal initilaiser
            layers.BatchNormalization(),
            layers.Activation(activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(128 ,activation = "relu", kernel_initializer = "uniform"),  #
            layers.BatchNormalization(),
            layers.Activation(activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(6, activation = "softmax", kernel_initializer = "uniform"),  #  check whether it needs activation or not.

    #         layers.Dense(2, activation="relu", name="layer1"),
    #         layers.Dense(3, activation="relu", name="layer2"),
    #         layers.Dense(4, name="layer3"),
        ]


    )
    model.compile(loss='categorical_crossentropy',
#                   optimizer=optimizers.SGD(learning_rate=2e-4),  # change it to adagrad (lr = 0.001, initial_accumulator_valur = 0.1, epsilon = 1e-07)
                  optimizer=optimizers.Adam(learning_rate=6e-4),  # change it to adagrad (lr = 0.001, initial_accumulator_valur = 0.1, epsilon = 1e-07)
#                   optimizer=optimizers.Adadelta(learning_rate=.1),  # change it to adagrad (lr = 0.001, initial_accumulator_valur = 0.1, epsilon = 1e-07)
                  metrics=['accuracy'])


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization_6 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
activation_6 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 224, 224, 128)     73856     
_________________________________________________________________
batch_normalization_7 (Batch (None, 224, 224, 128)     512       
_________________________________________________________________
activation_7 (Activation)    (None, 224, 224, 128)    

In [ ]:

result = model.fit_generator(
            trainGenerator,
            epochs = 100,
            steps_per_epoch = 500,
            validation_data = validation_generator,
            validation_steps = 432,
                workers = 5,
                use_multiprocessing = True,
)


In [ ]:
%matplotlib inline

In [ ]:
accu=result.history['accuracy']
#val_acc=res.history['val_accuracy']
loss=result.history['loss']
#val_loss=res.history['val_loss']

epochs=range(len(accu)) #No. of epochs

# graph plotting
import matplotlib.pyplot as plt
plt.plot(epochs,accu,'r',label='Training Accuracy')
plt.plot(epochs,val_acc,'g',label='Testing Accuracy')  
plt.legend()
plt.figure()


In [ ]:
#Plot training and validation loss per epoch
plt.plot(epochs,loss,'r',label='Training Loss')
plt.plot(epochs,val_loss,'g',label='Testing Loss')   
plt.legend()
plt.show()